In [1]:
import requests
import urllib.request
from bs4 import BeautifulSoup, NavigableString, Tag
import re

In [2]:
import lxml.html as lh
import pandas as pd

In [87]:
r = requests.get(url)
html_content = r.text
soup = BeautifulSoup(html_content, "html.parser")

all_labels = soup.find_all("table", class_="people")
# print(all_labels)
#cuts = soup.find_all("td", scope_="col")
res=[]

for label in all_labels:
    sub = re.sub(r"on leave", ' ', label.text)
    sub = re.sub(r"(?:\().*?(?:\))+", ' ', sub)
    sub = re.sub(r"(?:\d).*?(?:\n)+", ' ', sub)
    sub = re.sub(r"(?:[\:\,]).*?(?=\n)", '', sub)
    sub = re.sub(r"(?<=\n).(?=\n)", ' ', sub)
    sub = re.sub(r"\bdr\b", " ", sub)
    sub = re.sub(r"mgr", " ", sub)
    sub = re.sub(r"hab\.", " ", sub)
    sub = re.sub(r"prof\.", " ", sub)
    sub = re.sub(r"inż\.", " ", sub)
    sub = re.sub(r"(?<=\n)\s+(?=\w)", '', sub)
    sub = re.sub(r"\n+", ";", sub)
    sub = re.sub(r" ;", ";", sub)
    res.append(sub.split(';'))
    

res = [i for l in res for i in l if len(i) > 4]
res = [i.split(' ') for i in res]
res = [i[0] + ' ' + i[-1] for i in res if len(i[0])>2]
res = set(res)
res2 = []

pol_ang = {'ą':'a', 'ć':'c', 'ę':'e', 'ł':'l', 'ń':'n', 'ó':'o', 'ś':'s', 'ź':'z', 'ż':'z',
          'Ą':'A', 'Ć':'c', 'Ę':'E', 'Ł':'L', 'Ń':'N', 'Ó':'O', 'Ś':'S', 'Ź':'Z', 'Ż':'Z'}

for i in res:
    string = ''
    for l in i:
        if pol_ang.get(l):
            l = pol_ang[l]
        string += l
        
    res2.append(string)
            
emps = [i.split(' ') for i in res2]


In [91]:
pd.DataFrame(emps, columns=["name", "surname"]).to_csv("ph_emps.csv", index=False)